In [2]:
import csv
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.optim as optim

file_names = glob.glob("data/*.csv")
classes = ['Wipe', 'Sweep', 'Rub', 'Insert', 'Other']
batch_size = 256

train_data = []
train_label = []
num_traind = 0
for file in file_names:
    with open(file) as f:
        reader = csv.reader(f)
        num_dim = 0
        for row in reader:
            if num_dim == 0:
                num_data = len(train_data)
                train_data.append([[], [], []])
            train_data[num_data][0].append(row[2])  #x軸の加速度データを入れる
            train_data[num_data][1].append(row[3])  #y軸の加速度データを入れる
            train_data[num_data][2].append(row[4])  #z軸の加速度データを入れる
            num_dim += 1
            if num_dim == 200:
                index = classes.index(row[5])
                train_label.append(index)
                num_dim = 0
                num_traind += 1
                if ('nan' in train_data[num_data][0]) or ('nan' in train_data[num_data][1]) or ('nan' in train_data[num_data][2]):
                    train_data.pop(num_data)
                    train_label.pop(num_data)
                if 200 * num_traind >= 30000:
                    num_traind = 0
                    break

train_data = np.array(train_data, dtype=np.int64)
train_label = np.array(train_label, dtype=np.int64)

train_data = torch.tensor(train_data, dtype=torch.float32)
train_label = torch.tensor(train_label, dtype=torch.int64)
train_dataset = torch.utils.data.TensorDataset(train_data, train_label)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

ModuleNotFoundError: No module named 'torch'

In [ ]:
class LSTM(nn.Module):

    def __init__(self):

        super(LSTM, self).__init__()
        self.seq_len = 200 # XYZ軸を時系列のSequenceとしてLSTMに入力する
        self.feature_size = 3 # 1つの軸当たりの特徴量の次元としてLSTMに入力する
        self.hidden_layer_size = 100 # 隠れ層のサイズ
        self.lstm_layers = 1 # LSTMのレイヤー数　(LSTMを何層重ねるか)
        self.lstm = nn.LSTM(self.feature_size, self.hidden_layer_size, num_layers = self.lstm_layers)
        self.fc = nn.Linear(self.hidden_layer_size, 10)

    def init_hidden_cell(self, batch_size): # LSTMの隠れ層 hidden と記憶セル cell を初期化
        hedden = torch.zeros(self.lstm_layers, batch_size, self.hidden_layer_size, device=x.device)
        cell = torch.zeros(self.lstm_layers, batch_size, self.hidden_layer_size, device=x.device)
        return (hedden, cell)

    def forward(self, x):
        batch_size = x.shape[0]
        # print(batch_size)

        hedden = torch.zeros(self.lstm_layers, batch_size, self.hidden_layer_size, device=x.device)
        cell = torch.zeros(self.lstm_layers, batch_size, self.hidden_layer_size, device=x.device)
        self.hidden_cell = (hedden, cell)

        # print(x.size())
        x = x.view(batch_size, self.seq_len, self.feature_size)  # (Batch, Cannel, Height, Width) -> (Batch, Height, Width) = (Batch, Seqence, Feature)
                                                                 # 画像の Height を時系列のSequenceに、Width を特徴量の次元としてLSTMに入力する
        # print(x.size())
        x = x.permute(1, 0, 2)                                   # (Batch, Seqence, Feature) -> (Seqence , Batch, Feature)

        lstm_out, (h_n, c_n) = self.lstm(x, self.hidden_cell)    # LSTMの入力データのShapeは(Seqence, Batch, Feature)
                                                                 # (h_n) のShapeは (num_layers, batch, hidden_size)
        x = h_n[-1,:,:]                                          # lstm_layersの最後のレイヤーを取り出す  (B, h)
        x = self.fc(x)

        return x


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#モデル作成
net = LSTM().to(device)   # GPUを使用する場合のために明示的に .to(device) を指定
# nn.CrossEntropyLoss() はソフトマックス関数＋クロスエントロピー誤差
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
  print('Epoch {}/{}'.format(epoch+1, num_epochs))
  print('-------------')

  epoch_loss = 0.0  # epochの損失和
  epoch_corrects = 0  # epochの正解数
  epoch_train_loss = 0
  epoch_train_acc = 0

  net.train()
  # データローダーからミニバッチを取り出すループ
  for data in train_loader:

      # GPUを使用する場合は明示的に指定
      inputs = data[0].to(device)
      labels = data[1].to(device)

      # optimizerを初期化
      optimizer.zero_grad()

      # 順伝搬（forward）計算
      outputs = net(inputs)              # 順伝播
      loss = criterion(outputs, labels)  # 損失を計算
      _, preds = torch.max(outputs, 1)   # ラベルを予測
      print(preds[0].item())

      # バックプロパゲーション
      loss.backward()
      optimizer.step()

      # イタレーション結果の計算
      # lossの合計を更新
      # epoch_loss += loss.item() * inputs.size(0)
      # 正解数の合計を更新
      # epoch_corrects += torch.sum(preds == labels.data)
      epoch_train_loss += loss.item()/len(train_loader)
      acc = (outputs.argmax(dim=1) == labels).float().mean()
      epoch_train_acc += acc/len(train_loader)

  # epochごとのlossと正解率を表示
  # epoch_loss = epoch_loss / len(train_loader)
  # epoch_acc = epoch_corrects.double() / len(train_loader)

  net.eval()
  # print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
  print(f'Epoch {epoch+1} : train acc. {epoch_train_acc:.4f} train loss {epoch_train_loss:.4f}')

In [ ]:
model_path = 'model.pth'
torch.save(net.to('cpu').state_dict(), model_path)